Data File ==> DataFrame, Numpy (전처리) ==> Tensor ==> Dataset(피처 + 타겟) ==> DataLoader 생성

### Dataset & DataLoader 살펴보기
- Pytorch에서 배치크기만 데이터를 조절하기 위한 메카니즘
- Dataset : 사용 데이터를 기반으로 사용자 정의 클래스 작성
- DataLoader : 지정된 Dataset에서 지정된 batch size만큼 피처와 타겟을 추출하여 전달

[1] 모듈로딩 및 데이터 준비

In [60]:
### ===> 모듈 로딩
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd

[2] 데이터셋 생성

- 사용자 정의 데이터셋 생성

In [61]:
file = '../data/iris.csv'
irisDF = pd.read_csv(file, header = None)
irisDF.columns = ['sepal_length', 'sepal_width', 'petal_length','petal_width','variety']
irisDF

,sepal_length,sepal_width,petal_length,petal_width,variety
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [62]:
irisNP = np.loadtxt(file, delimiter = ',', usecols = [0,1,2,3])
irisNP

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [63]:
# 데이터의 타입 체크
type(irisDF), type(irisNP), irisDF.__class__.__name__, irisNP.__class__.__name__

(pandas.core.frame.DataFrame, numpy.ndarray, 'DataFrame', 'ndarray')

In [64]:
if irisDF.__class__.__name__ == 'DataFrame' :
    print('DF')
else:
    print('-----')

DF


In [65]:
print(isinstance(irisDF, pd.DataFrame), isinstance(irisNP, pd.DataFrame), isinstance(irisDF, np.ndarray),isinstance(irisNP, np.ndarray), sep = '\n')

True
False
False
True


In [66]:
### 사용자 정의 DataSet 클래스
# - 데이터의 Tensor 변환

class DLDataset(Dataset):
    # 초기화 함수 콜백함수 (callback function)
    def __init__(self, x_data, y_data):
        super().__init__()
        
        # x,y 데이터 ==> ndarray
        x_data = x_data.values if isinstance(x_data, pd.DataFrame) else x_data
        y_data = y_data.values if isinstance(y_data, pd.DataFrame) else y_data
        # 넘파이면 그대로 ㄱㅊ    다만 데이터프레임이면 values만 뺴서 갖고오자능
        
        # ndarray ==> tensor
        self.feature = torch.FloatTensor(x_data)
        self.target = torch.LongTensor(y_data)         # 라벨 인코딩 형식으로 진행함 (원핫인코딩 ㄴㄴ)
        
    # 데이터셋의 개수 체크 함수 콜백함수 (callback function)
    def __len__(self):
        return self.target.shape[0]
    
    # 특정 인덱스 데이터 + 라벨 반환 콜백함수 (callback function)
    def __getitem__(self, index):
        return self.feature[index], self.target[index]


In [67]:
## 피처와 라벨로 분리
featureDF = irisDF[irisDF.columns[:-1]]
targetDF = irisDF[irisDF.columns[-1]]

print(f"featureDF => {featureDF.shape} , {featureDF.ndim}D")
print(f"targetDF => {targetDF.shape} , {targetDF.ndim}D")

featureDF => (150, 4) , 2D
targetDF => (150,) , 1D


In [68]:
# object 타입 타겟 ===> int 타입 타겟 변환
from sklearn.preprocessing import LabelEncoder

targetNP = LabelEncoder().fit_transform(targetDF)
targetNP = targetNP.reshape(-1,1)
print(targetNP.shape, targetNP.ndim)

(150, 1) 2


In [69]:
# 데이터셋 생성  ==> DF, NP
my_dataset = DLDataset(featureDF, targetNP)

In [70]:
print(my_dataset[0], featureDF.iloc[0], targetDF[0], sep = '\n\n')

(tensor([5.1000, 3.5000, 1.4000, 0.2000]), tensor([0]))

sepal_length    5.1
sepal_width     3.5
petal_length    1.4
petal_width     0.2
Name: 0, dtype: float64

Iris-setosa


In [71]:
# 데이터셋 생성 => NP, NP
my_dataset2 = DLDataset(irisNP, targetNP)
print(my_dataset2[0])

(tensor([5.1000, 3.5000, 1.4000, 0.2000]), tensor([0]))


- [2-3] 학습용, 검증용, 테스트용 Dataset <hr>

In [72]:
### ===> 파이토치
from torch.utils.data import random_split

# 학습용, 검증용, 테스트 데이터 비율
seed = torch.Generator().manual_seed(2)

trainDS, validDS, testDS = random_split(my_dataset2, [0.7, 0.1, 0.2], generator = seed )

print(f"trainDS => {len(trainDS)}, validDS => {len(validDS)}, testDS => {len(testDS)}")

trainDS => 105, validDS => 15, testDS => 30


In [73]:
print(f"Subset 속성 =>\n indices: \n {trainDS.indices} \n dataset : {trainDS.dataset}")

Subset 속성 =>
 indices: 
 [48, 81, 63, 33, 30, 84, 24, 49, 10, 117, 40, 128, 89, 87, 115, 53, 35, 97, 80, 5, 20, 75, 59, 64, 43, 60, 90, 0, 127, 86, 125, 111, 26, 47, 120, 65, 107, 118, 21, 116, 148, 67, 124, 70, 55, 46, 15, 18, 29, 8, 132, 22, 106, 101, 144, 133, 51, 9, 4, 85, 32, 78, 23, 41, 136, 14, 36, 140, 142, 17, 71, 68, 98, 108, 103, 3, 145, 141, 39, 146, 54, 109, 93, 28, 147, 58, 112, 149, 38, 61, 104, 52, 122, 44, 138, 31, 6, 42, 69, 134, 130, 66, 110, 12, 139] 
 dataset : <__main__.DLDataset object at 0x000001D2B2A90DF0>


In [74]:
print(f"Subset 속성 =>\n indices: \n {validDS.indices} \n dataset : {validDS.dataset}")

Subset 속성 =>
 indices: 
 [77, 102, 11, 19, 7, 143, 123, 25, 129, 34, 2, 16, 27, 74, 83] 
 dataset : <__main__.DLDataset object at 0x000001D2B2A90DF0>


[3] DataLoader 생성 : 학습용, 검증용, 테스트용 <hr>

In [75]:
# DataLoader 생성
# drop_last 매개변수 : 배치 사이즈로 데이터셋 분리 후 남는 데이터 처리 방법 설정 [기본 : False]
batch = 10
trainDL = DataLoader(trainDS, batch_size = batch)
validDL = DataLoader(validDS, batch_size = batch)
testDL = DataLoader(testDS, batch_size = batch)

len(trainDL), len(validDL), len(testDL)

(11, 2, 3)

In [76]:
# Epoch당 반복 단위
print(f'batch_size : {batch}')
print(f"trainDS => {len(trainDS)}개, validDS => {len(validDS)}개, trainDS => {len(trainDS)}개")
print(f"trainDL => {len(trainDL)}개, validDL => {len(validDL)}개, testDL => {len(testDL)}개")

batch_size : 10
trainDS => 105개, validDS => 15개, trainDS => 105개
trainDL => 11개, validDL => 2개, testDL => 3개


[4] Model 클래스 정의 : 입/출력 피처수, 층 수, 은닉층의 노드수 <hr>
- 구조설계
    * 입력층 : 입력 <= 피처 갯수, iris 4개 
    * 은닉층 : 마음대로 알아서 잘
    * 출력층 : 출력 <= [분류] 타겟 클래스 갯수  [회귀] 1개

In [77]:
# 모델 클래스 정의
# 클래스명 : CModel

class CModel(nn.Module):    # nn.Module을 상속받는다.
    
    # 모델 구성 요소 정의 함수
    def __init__(self, in_, out_):
        super().__init__()
        self.input_layer = nn.Linear(in_, 100)
        self.relu = nn.ReLU()
        self.hidden_layer = nn.Linear(100, 27)
        self.output_layer = nn.Linear(27, out_)
        
    # 순방향 학습 진행
    def forward(self, x):
        x = self.input_layer(x)            # W1x1 + W2x2 _ ... + Wnxn + b 반환
        x = self.relu(x)                   # relu 함수 결과 100개 반환
        x = self.hidden_layer(x)
        x = self.relu(x)
        x = self.output_layer(x)
        return x

[5] 학습 준비 : 실행 디바이스, 모델, 최적화, 손실함수, 학습 횟수, 학습 함수, 평가 함수, 예측 함수 <hr>

In [78]:
# 실행 디바이스 설정
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# 학습 횟수
EPOCHS = 50

In [79]:
# 모델 인스턴스
IN, OUT = my_dataset2.feature.shape[1], len(np.unique(targetDF))
print(IN, OUT)
model = CModel(IN, OUT).to(DEVICE)

4 3


In [80]:
# 손실 함수
LOSS_FN = nn.CrossEntropyLoss().to(DEVICE)

# 최적화 인스턴스
import torch.optim as optim
OPTIMIZER = optim.Adam(model.parameters())

- 학습 및 검증 관련 함수 정의

In [81]:
### ===> 학습 진행 함수
def training():
    # 학습 모드 => 정규화, 경사하강법, 드랍아웃 등의 기능 활성화
    model.train()
    
    # 배치 크기만큼 학습 진행 및 저장
    train_loss = []
    for cnt, (feature, target) in enumerate(trainDL):
        # print(cnt, feature, target)
        # 배치 크기만큼의 학습 데이터 준비      
        feature, target = feature.to(DEVICE), target.to(DEVICE)
        
        target = target.squeeze()
        
        # 학습
        pre_target = model(feature)
        print(f"pre_target => {pre_target.shape}, {pre_target.ndim}D")
        print(f"target => {target.shape}, {target.ndim}D")
        
        
        # 손실 계산
        loss = LOSS_FN(pre_target, target)
        train_loss.append(loss)
        
        # W, b 업데이트
        OPTIMIZER.zero_grad()
        loss.backward()
        OPTIMIZER.step()
        
        # 배치 단위 학습 진행 메시지 출력
        print(f"[Train {cnt} batch LOSS] ===> {loss}")
        
    # 에포크 단위 학습 진행 메시지 출력
    print(f"[Train loss] ===> {loss}")
    
    return train_loss

In [82]:
### ===> 검증 및 평가 진행 함수
def testing():
    pass

In [83]:
### ===> 예측 함수
def predict():
    pass

[6] 학습 진행 <hr>

In [84]:
for eps in range(EPOCHS):
    # 학습
    train_loss = training()
    
    # 검증
    # testing()
    
    print(f"[{eps}/{EPOCHS}] {sum(train_loss)/len(train_loss)}")

pre_target => torch.Size([10, 3]), 2D
target => torch.Size([10]), 1D
[Train 0 batch LOSS] ===> 0.983521580696106
pre_target => torch.Size([10, 3]), 2D
target => torch.Size([10]), 1D
[Train 1 batch LOSS] ===> 1.1552753448486328
pre_target => torch.Size([10, 3]), 2D
target => torch.Size([10]), 1D
[Train 2 batch LOSS] ===> 1.0536985397338867
pre_target => torch.Size([10, 3]), 2D
target => torch.Size([10]), 1D
[Train 3 batch LOSS] ===> 1.3102928400039673
pre_target => torch.Size([10, 3]), 2D
target => torch.Size([10]), 1D
[Train 4 batch LOSS] ===> 0.972711443901062
pre_target => torch.Size([10, 3]), 2D
target => torch.Size([10]), 1D
[Train 5 batch LOSS] ===> 1.1456689834594727
pre_target => torch.Size([10, 3]), 2D
target => torch.Size([10]), 1D
[Train 6 batch LOSS] ===> 0.9869710206985474
pre_target => torch.Size([10, 3]), 2D
target => torch.Size([10]), 1D
[Train 7 batch LOSS] ===> 1.1264712810516357
pre_target => torch.Size([10, 3]), 2D
target => torch.Size([10]), 1D
[Train 8 batch LOSS] 

In [85]:
# DataLoader 속성
for _ , (feature, target) in enumerate(trainDL):
    print(f"[{_}] feature {feature.shape}")
    ## 로더에서 가지고온 데이터만큼 학습 진행 

[0] feature torch.Size([10, 4])
[1] feature torch.Size([10, 4])
[2] feature torch.Size([10, 4])
[3] feature torch.Size([10, 4])
[4] feature torch.Size([10, 4])
[5] feature torch.Size([10, 4])
[6] feature torch.Size([10, 4])
[7] feature torch.Size([10, 4])
[8] feature torch.Size([10, 4])
[9] feature torch.Size([10, 4])
[10] feature torch.Size([5, 4])
